In [ ]:
#!L
import pandas as pd
import numpy as np
import os
import sys

PROJ_PATH = "../.."
SRC_PATH = os.path.join(PROJ_PATH, "intent_complex_model")
sys.path.append(SRC_PATH)

import random
import time
from sklearn.model_selection import train_test_split
import nltk

from src.data.handle_symbols import handle
from src.data.text_augmentation import augment_df

random.seed(0)
np.random.seed(0)
pd.__version__

In [25]:
#!L
DATA_PATH = os.path.join(SRC_PATH, "data")
RAW_PATH = os.path.join(DATA_PATH, "raw")
PROCESSED_PATH = os.path.join(DATA_PATH, "processed")
MODELS_PATH = os.path.join(SRC_PATH, "models")

In [26]:
#!L
train = pd.read_excel(os.path.join(RAW_PATH, 'train_88-6-6.xlsx'))[['phrases', 'intent']].reset_index(drop=True)
valid = pd.read_excel(os.path.join(RAW_PATH, 'validate_88-6-6.xlsx'), engine='openpyxl')[['phrase', 'intent']].reset_index(drop=True)
train.columns, valid.columns = [['text', 'intent'] for _ in range(2)]
train.shape, valid.shape

((93730, 2), (8240, 2), (8092, 2))

In [13]:
test = test[test.intent.isin(train.intent.unique())]
valid = valid[valid.intent.isin(train.intent.unique())]

In [14]:
train.shape

(93730, 2)

In [15]:
valid.intent.nunique(), test.intent.nunique()

(94, 94)

In [17]:
#!L
very_small_intents = list(train.intent.value_counts()[train.intent.value_counts()<30].index)
med_small_intents = list(train.intent.value_counts()[(train.intent.value_counts()>=30)&(train.intent.value_counts()<100)].index)
small_intents = list(train.intent.value_counts()[(train.intent.value_counts()>=100)&(train.intent.value_counts()<200)].index)
big_intents = list(train.intent.value_counts()[(train.intent.value_counts()>=200)].index)
print(len(very_small_intents), len(med_small_intents), len(small_intents), len(big_intents))

37 57 19 103


In [18]:
print(time.ctime())
vs = augment_df(train[train.intent.isin(very_small_intents)], frac=10, bert_path=None)
print(time.ctime())
ms = augment_df(train[train.intent.isin(med_small_intents)], frac=2.5, bert_path=None)
print(time.ctime())
ms = augment_df(ms, frac=2, bert_path=BERT_PATH)
print(time.ctime())
s = augment_df(train[train.intent.isin(small_intents)], frac=3, bert_path=BERT_PATH)
print(time.ctime())
b = train[train.intent.isin(big_intents)].copy()

train = pd.concat([vs, ms, s, b])
train.shape

Wed Sep  1 16:05:02 2021
Wed Sep  1 16:05:02 2021
Wed Sep  1 16:05:02 2021
Wed Sep  1 16:06:34 2021
Wed Sep  1 16:07:35 2021


(115182, 2)

In [19]:
train = train[~train.text.isin(valid.text)]
train = train[~train.text.isin(test.text)]
train.drop_duplicates(inplace=True)

In [20]:
#!L
train.shape

(103832, 2)

In [21]:
#!M
train.to_csv(os.path.join(PROCESSED_PATH, 'intents_train_context.csv'), index=False)
valid.to_csv(os.path.join(PROCESSED_PATH, 'intents_valid.csv'), index=False)